# Design

In [ ]:
%load_ext autoreload
%autoreload 2

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '3mm'
design.chips.main.size['size_y'] = '2.4mm'
design.chips.main.size.center_x = '0.5mm'
design.chips.main.size.center_y = '0mm'

gui = MetalGUI(design)

design.delete_all_components()

q1 = TransmonPocket(design, 'Q1', options = dict(
    pad_width = '425 um', 
    pocket_height = '625um',
    pocket_width = '500um',
    orientation = 0,
    hfss_inductance ='14nH',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))
otg = OpenToGround(design, 'open_to_ground', options=dict(pos_x='1.75mm',  pos_y='0um', orientation='0'))
readout = RouteMeander(design, 'readout',  Dict(
            total_length='6 mm',
            hfss_wire_bonds = True,
            fillet='90 um',
            lead = dict(start_straight='100um'),
            pin_inputs=Dict(
                start_pin=Dict(component='Q1', pin='readout'),
                end_pin=Dict(component='open_to_ground', pin='open')), ))

gui.rebuild()
gui.autoscale()

# Eigenmode Simulation

In [4]:
from PALACE_Simulation import PALACE_Simulation

#Eigenmode Simulation Options
eigen_user_options = {
            "mesh_refinement":  0,                                         #refines mesh in PALACE - essetially divides every mesh element in half
            "dielectric_material": "silicon",                              #choose dielectric material - 'silicon' or 'sapphire'
            "starting_freq": 4.5,                                          #starting frequency in GHz 
            "number_of_freqs": 3,                                          #number of eigenmodes to find
            "solns_to_save": 3,                                            #number of electromagnetic field visualizations to save
            "solver_order": 2,                                             #increasing solver order increases accuracy of simulation, but significantly increases sim time
            "solver_tol": 1.0e-8,                                          #error residual tolerance foriterative solver
            "solver_maxits": 100,                                          #number of solver iterations
            "mesh_max": 170e-3,                                            #maxiumum element size for the mesh in mm
            "mesh_min": 6e-3,                                             #minimum element size for the mesh in mm
            "mesh_sampling": 150,                                          #number of points to mesh along a geometry
            "sim_directory": 'C:\\PALACE_Simulations\\'                    #parent directory where all simulation files will be saved
}

HPC_options = {
            "hpc_nodes": '3',
            "cpus_per_node": '25',
            "sim_time": '3:30:00',
            "sim_memory": '250G',
            "account_name": 'a_fedorov',
            "palace_location": '/scratch/project/palace-sqdlab/Palace-Project-NEW/palace/build_v0.12/bin/palace-x86_64.bin',
            "input_files_location": '/scratch/project/palace-sqdlab/inputFiles/',
            "output_files_location": '/scratch/project/palace-sqdlab/outputFiles/'
}

sim = PALACE_Simulation('Eigenmode', 'eigen_epr_wide_cap', design, eigen_user_options, ports = [], hpc_options = HPC_options)
sim.run_simulation()

Creating geometry for Eigenmode Simulation.
No ports for processing.
<class 'float'>
Geometry successfully built in Gmsh.
Finely meshing surfaces: [7]
Mesh successfully built in Gmsh.
Directory 'eigen_epr_wide_cap' created at 'C:\PALACE_Simulations\eigen_epr_wide_cap'.
Simulation files created.


# Capacitance Simulation

In [ ]:
from PALACE_Simulation import PALACE_Simulation

#Eigenmode Simulation Options
cap_user_options = {
            "mesh_refinement":  0,                                         #refines mesh in PALACE - essetially divides every mesh element in half
            "dielectric_material": "silicon",                              #choose dielectric material - 'silicon' or 'sapphire'
            "solns_to_save": 2,                                            #number of electromagnetic field visualizations to save
            "solver_order": 2,                                             #increasing solver order increases accuracy of simulation, but significantly increases sim time
            "solver_tol": 1.0e-8,                                          #error residual tolerance foriterative solver
            "solver_maxits": 100,                                          #number of solver iterations
            "mesh_max": 160e-3,                                            #maxiumum element size for the mesh in mm
            "mesh_min": 10e-3,                                             #minimum element size for the mesh in mm
            "mesh_sampling": 120,                                          #number of points to mesh along a geometry
            "sim_directory": 'C:\\PALACE_Simulations\\'                    #parent directory where all simulation files will be saved
}

HPC_options = {
            "hpc_nodes": '3',
            "cpus_per_node": '20',
            "sim_time": '2:00:00',
            "sim_memory": '250G',
            "account_name": 'a_fedorov',
            "palace_location": '/scratch/project/palace-sqdlab/Palace-Project-NEW/palace/build_v0.12/bin/palace-x86_64.bin',
            "input_files_location": '/scratch/project/palace-sqdlab/inputFiles/',
            "output_files_location": '/scratch/project/palace-sqdlab/outputFiles/'
}

sim = PALACE_Simulation('Capacitance', 'cap_transmon_res', design, cap_user_options, ports = [], hpc_options = HPC_options)
sim.run_simulation()

Creating geometry for Capacitance Simulation.
No ports for processing.
Geometry successfully built in Gmsh.
Finely meshing surfaces: [7]
Mesh successfully built in Gmsh.
Directory 'cap_transmon_res' created at 'C:\PALACE_Simulations\cap_transmon_res'.
Simulation files created.


In [3]:
import gmsh
gmsh.fltk.run()